In [ ]:
!kaggle datasets download -d snap/amazon-fine-food-reviews

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
!unzip /content/amazon-fine-food-reviews.zip

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
con = sqlite3.connect('/content/database.sqlite')

In [ ]:
df = pd.read_sql_query("""SELECT * FROM Reviews WHERE Score!=3 LIMIT 5000""",con)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
filter = lambda x: 0 if x<3 else 1
df["Score"] = df["Score"].map(filter)
df.head(3)

In [ ]:
df['Score'].value_counts()

In [ ]:
uc = pd.read_sql_query("""
select *,count(*) from Reviews where Score!=3 group by UserId having count(*)>1

""",con)
uc.head()

In [ ]:
uc[uc['count(*)']==uc['count(*)'].max()]

In [ ]:
df[df['UserId'] == "A3OXHLG6DIBRW8"].shape

In [ ]:
df[df['UserId'].isin((df[df["Time"].duplicated()]["UserId"].value_counts() > 1).index)]

In [ ]:
df[df["UserId"] == "AR5J8UI46CURR"]["Text"]

In [ ]:
cdf = pd.read_sql_query("""
select * from Reviews where Score != 3
""",con)

In [ ]:
qq = cdf.groupby(['Time','UserId',"ProductId"])

In [ ]:
qq.count()[qq.count()["ProfileName"]==11]

In [ ]:
cdf[cdf["UserId"] == "A29JUMRL1US6YP"].sort_values(["Time","ProductId"])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
documents = ["The quick brown fox jumps over the lazy dog",
             "The dog is slow and the fox is very quick"]
vectorizer = CountVectorizer()
mat = vectorizer.fit_transform(documents)
# matrix = vectorizer.transform(documents)


In [ ]:
print(mat,type(mat))

In [ ]:
mat.shape

In [ ]:

complete = [i.lower().split(" ") for i in documents]
complete

In [ ]:
store = {}
for i in documents:
  for j in i.lower().split(' '):
    store.setdefault(j,0)
    store[j]+=1

In [ ]:
words = list(store.keys())
words

In [ ]:
indeces = {words[i]:i for i in range(len(words))}
indeces

In [ ]:
import numpy as np
a = np.zeros((len(documents),len(words)),dtype=int)
ip = 0
for i in documents:
  for j in i.lower().split(" "):
    a[ip][indeces[j]] +=1
  ip+=1


In [ ]:
a

In [ ]:
print(mat,mat.get_shape())

In [ ]:
class MyCountVectorizer:
    def __init__(self):
        self.vocabulary_ = {}  # To store word counts and their indices

    def fit_transform(self, documents):
        # Tokenize documents (split into words)
        tokenized_docs = []
        for doc in documents:
            tokenized_docs.append(doc.lower().split())  # Lowercase and split

        # Build vocabulary (unique words and their counts)
        vocabulary = {}
        for doc in tokenized_docs:
            for word in doc:
                if word not in vocabulary:
                    vocabulary[word] = 0
                vocabulary[word] += 1

        # Assign unique indices to words in the vocabulary
        self.vocabulary_ = {word: i for i, word in enumerate(vocabulary)}

        # Create a sparse matrix to store term frequencies (replace with actual sparse matrix implementation)
        term_frequency_matrix = []
        for doc in tokenized_docs:
            frequency_vector = [0] * len(self.vocabulary_)  # Initialize with zeros
            for word in doc:
                if word in self.vocabulary_:
                    frequency_vector[self.vocabulary_[word]] += 1  # Increment count for each word
            term_frequency_matrix.append(frequency_vector)

        return term_frequency_matrix

# Example usage
documents = ["The quick brown fox jumps over the lazy dog",
              "The dog is slow and the fox is very quick"]

vectorizer = MyCountVectorizer()
term_frequency_matrix = vectorizer.fit_transform(documents)

# Print the term frequency matrix (replace with actual sparse matrix access methods)
for doc in term_frequency_matrix:
    print(doc)


In [ ]:
def give_counts():
  count = []
  ind= {}
  words = []
  id=0
  for i in documents:
    ss = {}
    for j in i.lower().split(" "):
      ind.setdefault(j,-1)
      if ind[j] == -1:
        ind[j] = id
        words.append(j)
        id+=1
      ss.setdefault(ind[j],0)
      ss[ind[j]]+=1

    count.append(ss)


  return words,count,ind



In [ ]:
w,c,inx = give_counts()
print(w,inx)

for i in range(len(c)):
  for k,v in c[i].items():
    print(f"({i},{k}) {v}")
  print("--"*10)




In [ ]:
#Multy Thredding

In [ ]:
from threading import *
import time


l = Lock()
def function(name):
  l.acquire()
  for i in range(10):
    print("Good Morning ",end="")
    time.sleep(2)
    print(name)
  l.release()

th1 = Thread(target=function,args=("Youraj",))
th2 = Thread(target=function,args=("Dhonei",))

th1.start()
th2.start()

# for i in range(10):
#   print("Main Thred {}".format(i))

In [ ]:
import requests
from tqdm import tqdm


def download_file(url, filename):
  """Downloads a file from the specified URL and saves it with the given filename,
  displaying a progress bar during the download.

  Args:
      url: The URL of the file to download.
      filename: The name of the file to save the downloaded content as.
  """

  # Check if the 'requests' and 'tqdm' libraries are installed
  try:
    requests.get('https://httpbin.org/get')
    from tqdm import tqdm  # Import tqdm within the function to avoid potential errors
  except ModuleNotFoundError as e:
    print(f"The following libraries are not installed: {', '.join(e.args)}")
    print("Please install them using 'pip install requests tqdm'.")
    return

  # Prompt user for URL and filename (avoid providing a specific URL)
  url = "https://pipeline-2.hotstorage.life/cfe85b1b9416b5ab38b0b684ed79a3284d61487474c6443c58e1030e65f0548f06635e7334b7745870f49d86f912c95609b4ba84ecf7b1ef6e131cb15a118e3a9d501059905e2e71cec7ae36bea5e03dbcf796715a74e63b571ae2d4f4bdada107b3eeaa036ebda032750f53d1ffd2d9c87adae06d7566a4c55c6baa8e21b2a392b4f21b731ceba0550046cee3f45c094216174340355b4908c980575b0d950a77bf04876efebe92ec151a67942e1ccb::95d3669530b51f2fbc5885221cbdb25a/Hanuman.2024.2160p.Jio.WEB-DL.DD5.1.H.264-Spidey.mkv"
  filename = "hanuman"

  try:
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise an exception for unsuccessful requests

    # Get file size (if available) for progress bar accuracy
    file_size = int(response.headers.get('content-length', None))

    # Create a progress bar with meaningful description
    with tqdm(total=file_size, unit='B', unit_scale=True, unit_divisor=1024, desc=f"Downloading: {filename}") as progress_bar:
      with open(filename, 'wb') as f:
        for chunk in response.iter_content(1024):
          if chunk:  # filter out keep-alive new chunks
            f.write(chunk)
            progress_bar.update(len(chunk))  # Update progress bar based on chunk size

    print(f"MKV file downloaded successfully as '{filename}'.")
  except requests.exceptions.RequestException as e:
    print(f"An error occurred while downloading the file: {e}")


if __name__ == "__main__":
  download_file(url="", filename="")


In [ ]:
!apt-get update && apt-get install -y ffmpeg


In [ ]:
!ffprobe -v error -select_streams v:0 -show_entries stream=codec_name,bit_rate /content/hanuman.mkv


In [ ]:
!ffmpeg -i /content/hanuman.mkv -c:v libx264 -c:a copy -vf scale=1920:1080 output_1080p.mkv


In [ ]:
!ffprobe -v error -select_streams a: -show_entries stream=codec_name,channels,sample_rate /content/hanuman.mkv
